In [ ]:
#@title Download L5 Sample Dataset and install L5Kit
import os
RunningInCOLAB = 'google.colab' in str(get_ipython())
if RunningInCOLAB:
    !wget https://raw.githubusercontent.com/lyft/l5kit/master/examples/setup_notebook_colab.sh -q
    !sh ./setup_notebook_colab.sh
    os.environ["L5KIT_DATA_FOLDER"] = open("./dataset_dir.txt", "r").read().strip()
else:
    os.environ["L5KIT_DATA_FOLDER"] = "/data/parth-data/Level5_WP/"
    print("Not running in Google Colab.")

In [ ]:
import matplotlib.pyplot as plt

import numpy as np

from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset

from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points
from tqdm import tqdm
from collections import Counter
from l5kit.data import PERCEPTION_LABELS
from prettytable import PrettyTable

import os

from l5kit.visualization.visualizer.zarr_utils import zarr_to_visualizer_scene
from l5kit.visualization.visualizer.visualizer import visualize
from bokeh.io import output_notebook, show
from l5kit.data import MapAPI

In [ ]:
# Dataset is assumed to be on the folder specified
# in the L5KIT_DATA_FOLDER environment variable
os.environ["L5KIT_DATA_FOLDER"] = "/data/parth-data/Level5_WP/"
# get config
cfg = load_config_data("./visualisation_config.yaml")
print(cfg)

In [ ]:
print(f'current raster_param:\n')
for k,v in cfg["raster_params"].items():
    print(f"{k}:{v}")

In [ ]:
dm = LocalDataManager()
dataset_path = dm.require(cfg["val_data_loader"]["key"])
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()
print(zarr_dataset)

In [ ]:
from l5kit.environment.utils import get_scene_types, get_scene_types_as_dict
from l5kit.dataset import EgoDataset

rasterizer = build_rasterizer(cfg, dm)


scene_id_to_type_mapping_file = "../../dataset_metadata/train_split_1350.csv"
scene_id_to_type_val_path = "../../dataset_metadata/val_split_1350.csv"

# Group Structures
scene_type_to_id_dict = get_scene_types_as_dict(scene_id_to_type_mapping_file)
scene_id_to_type_list = get_scene_types(scene_id_to_type_mapping_file)

train_dataset_original = EgoDataset(cfg, zarr_dataset, rasterizer)
cumulative_sizes = train_dataset_original.cumulative_sizes


In [ ]:
# Loop over scenes
total_frames = cumulative_sizes[-1]
cumulative_sizes = np.insert(cumulative_sizes, 0, 0)
filter_frame_ids = [False] * total_frames
for index in range(len(cumulative_sizes)-1):
    # Determine boundaries
    start_frame = cumulative_sizes[index]
    end_frame = cumulative_sizes[index+1]
    len_scene = end_frame - start_frame
    if scene_id_to_type_list[index][0] == 'upper':
        filter_frame_ids[start_frame : end_frame] = [True] * len_scene    
filtered_frames = [i for i, x in enumerate(filter_frame_ids) if x]

In [ ]:
frames = zarr_dataset.frames[filtered_frames]
coords = np.zeros((len(frames), 2))
for idx_coord, idx_data in enumerate(tqdm(range(10000), desc="getting centroid to plot trajectory")):
    frame = frames[idx_data]
    coords[idx_coord] = frame["ego_translation"][:2]


plt.scatter(coords[:, 0], coords[:, 1], marker='.')
axes = plt.gca()
axes.set_xlim([-2500, 1600])
axes.set_ylim([-2500, 1600])